In [ ]:
import os
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.colors import Normalize
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib.transforms import Bbox
from shapely.geometry import Polygon, MultiPolygon
from PIL import Image
import pickle
import requests
import folium
from countryinfo import CountryInfo
from geopy.geocoders import Nominatim

In [ ]:
with open(r"HighRankedMovies_ByCountry.pickle", "rb") as input_file:
    movies = pickle.load(input_file)

In [ ]:
def select_entry(data):

    Selected_Films = {}

    for country in data:
        max_popularity_entry = max(data[country]['movies'], key= lambda x: x.get('vote_average'))
        # Filter out entries where the country in question is not the primary film.
        if max_popularity_entry['country'][0] == country:
            selected_entry = max_popularity_entry.copy()
        else:
            filtered_data = [entry for entry in data[country]['movies'] if entry['country'][0] == country]
            if filtered_data:
                max_popularity_entry = max(filtered_data, key=lambda x: x.get('vote_average')).copy()
        Selected_Films[country] = max_popularity_entry
    return Selected_Films

Movies2Plot = select_entry(movies)

In [ ]:
def RetrieveImages(movies):

    base_path = 'https://image.tmdb.org/t/p/original'

    for country in movies:
        img_data = requests.get(base_path + movies[country]['poster_path']).content

        with open('Posters/' + country + '.jpg', 'wb') as handler:
            handler.write(img_data)

# Save movie posters to the Posters directory
RetrieveImages(Movies2Plot)

In [ ]:
def create_image_annotation(ax, image_path, coordinates, zoom=1):
    image = plt.imread(image_path)

    if isinstance(coordinates, MultiPolygon):
        coords = coordinates.convex_hull.exterior.coords.xy
    elif isinstance(coordinates, Polygon):
        coords = coordinates.exterior.coords.xy

    ax.imshow(image, extent=(min(coords[0]), max(coords[0]), min(coords[1]), max(coords[1])), aspect='auto', zorder=2)

In [ ]:


# Load the world map data
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

fig, ax = plt.subplots(figsize=(15, 10))

# Loop through each country and plot it
for idx, row in world.iterrows():
    country_name = row['name']
    geometry = row['geometry']
    boundary = geometry.boundary

    if not os.path.exists(f'Posters/{country_name}.jpg'):
        print("No film listed for ", country_name)
        continue

    if geometry.geom_type == 'MultiPolygon':
        coordinates = geometry
    else:
        coordinates = geometry

    # Example: Use a sample JPEG image for each country
    image_path = f'Posters/{country_name}.jpg'

    create_image_annotation(ax, image_path, coordinates, zoom=0.2)

# Set plot limits
ax.set_xlim([-180, 180])
ax.set_ylim([-60, 90])

# Hide the axes
ax.axis('off')

# Show the plot
plt.show()

In [ ]:
import folium
from countryinfo import CountryInfo
import json
import requests
from shapely.geometry import shape

In [ ]:
tiles = "https://cartodb-basemaps-{s}.global.ssl.fastly.net/dark_all/{z}/{x}/{y}.png"

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

m = folium.Map(location=[0, 0], zoom_start=2, tiles=tiles, attr="CartoDB Dark Matter")

folium.GeoJson(
    world,
    name='geojson'
).add_to(m)

for idx, row in world.iterrows():
    country_name = row['name']
    geometry = row['geometry']
    boundary = geometry.boundary
    
    if not os.path.exists(f'Posters/{country_name}.jpg'):
        print("No film listed for ", country_name)
        continue
    
    geolocator = Nominatim(user_agent="geoapiExercises")
    location = geolocator.geocode(country_name)

    # Extract mainland boundaries using geopandas
    world_country = world[world['name'] == country_name]
    #mainland_geojson = world_country.geometry.simplify(tolerance=0.001).to_json()
    mainland_geojson = world_country.geometry.to_json()

    # Get bounding box coordinates for the mainland
    mainland_bounds = gpd.GeoDataFrame.from_features(json.loads(mainland_geojson)['features']).total_bounds
    
    folium.raster_layers.ImageOverlay(
        image = f'Posters/{country_name}.jpg',
        bounds=[[mainland_bounds[1], mainland_bounds[0]], [mainland_bounds[3], mainland_bounds[2]]],
        opacity=1.0,
        name=country_name
    ).add_to(m)

m.save('world_map_with_images.html')

In [ ]:
m